# Stats
public score: 0.7220902612826603
 
오차행렬:

 [[  878    92]
 
 [  511 10379]]

정확도: 0.9492

정밀도: 0.6321

재현율: 0.9052

F1: 0.7444

# 영업 성공 여부 분류 경진대회

## 1. 데이터 확인

### 필수 라이브러리

In [19]:
!pip install catboost

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [20]:
import pandas as pd
import numpy as np
from sklearn.metrics import (
    make_scorer,
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    StratifiedKFold
)
from sklearn.tree import DecisionTreeClassifier

from catboost import CatBoostClassifier

### 데이터 셋 읽어오기

In [21]:
df_train = pd.read_csv("train.csv") # 학습용 데이터
df_test = pd.read_csv("submission.csv") # 테스트 데이터(제출파일의 데이터)

In [22]:
df_train.head() # 학습용 데이터 살펴보기

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,1.0,/Quezon City/Philippines,AS,0.066667,32160,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Engineering,0,True
1,1.0,/PH-00/Philippines,AS,0.066667,23122,End-Customer,Enterprise,12.0,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Advertising,1,True
2,1.0,/Kolkata /India,AS,0.088889,1755,End-Customer,Enterprise,144.0,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,Construction,2,True
3,1.0,/Bhubaneswar/India,AS,0.088889,4919,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,IT/Software,3,True
4,1.0,/Hyderabad/India,AS,0.088889,17126,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,0,0,0.003079,0.026846,corporate / office,NaN,4,True


## 2. 데이터 전처리

In [23]:
# 레이블 인코딩할 칼럼들
label_columns = [
    "customer_country",
    "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_country.1",
    "customer_position",
    "response_corporate",
    "expected_timeline",
]

special_characters = ['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']

def remove_special_characters(text):
    for char in special_characters:
        text = text.replace(char, '')
    return text

# 범주형 특성은 모두 공백, 특수문자 제거 후 대문자 변환
for col in label_columns:
    df_train[col] = df_train[col].apply(
        lambda x: x if pd.isna(x) else remove_special_characters(x.replace(' ', '')).upper()
    )
    df_test[col] = df_test[col].apply(
        lambda x: x if pd.isna(x) else remove_special_characters(x.replace(' ', '')).upper()
    )

### 레이블 인코딩

In [24]:
def label_encoding(series: pd.Series) -> pd.Series:
    """범주형 데이터를 시리즈 형태로 받아 숫자형 데이터로 변환합니다."""

    my_dict = {}

    # 모든 요소를 문자열로 변환
    series = series.astype(str)

    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)

    return series

In [25]:
# 레이블 인코딩할 칼럼들
label_columns = [
    "customer_country",
    "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_country.1",
    "customer_position",
    "response_corporate",
    "expected_timeline",
]

df_all = pd.concat([df_train[label_columns], df_test[label_columns]])

for col in label_columns:
    df_all[col] = label_encoding(df_all[col])

다시 학습 데이터와 제출 데이터를 분리합니다.

In [26]:
for col in label_columns:  
    df_train[col] = df_all.iloc[: len(df_train)][col]
    df_test[col] = df_all.iloc[len(df_train) :][col]

### 2-2. 학습, 검증 데이터 분리

In [27]:
used_features = [
    'bant_submit', 
    # 'customer_country', 
    'business_unit',
    'com_reg_ver_win_rate', 
    'customer_idx', 
    'customer_type', 
    'enterprise',
    'historical_existing_cnt', 
    'id_strategic_ver', 
    'it_strategic_ver',
    'idit_strategic_ver', 
    'customer_job', 
    'lead_desc_length',
    'inquiry_type', 
    'product_category', 
    'product_subcategory',
    'product_modelname', 
    # 'customer_country.1', 
    'customer_position',
    'response_corporate', 
    'expected_timeline', 
    'ver_cus', 
    'ver_pro',
    'ver_win_rate_x', 
    'ver_win_ratio_per_bu', 
    'business_area',
    'business_subarea', 
    'lead_owner', 
    # 'is_converted'
]

In [28]:
x_train, x_val, y_train, y_val = train_test_split(
    df_train[used_features],
    df_train["is_converted"],
    test_size=0.2,
    shuffle=True,
    stratify=df_train['is_converted'],
    random_state=42,
)

## 3. 모델 학습

### 모델 정의 

In [29]:
train_df_vc = df_train['is_converted'].value_counts()
neg_pos_ratio = train_df_vc.iloc[0] / train_df_vc.iloc[1]

model = CatBoostClassifier(
    iterations=520,
    colsample_bylevel=0.8898455001363846,
    scale_pos_weight=10.191372142678775,
    random_seed=42,
    early_stopping_rounds=100,
    cat_features=[
        # used_features.index('customer_country'),
        used_features.index('business_unit'),
        used_features.index('customer_type'),
        used_features.index('enterprise'),
        used_features.index('customer_job'),
        used_features.index('inquiry_type'),
        used_features.index('product_category'),
        used_features.index('product_subcategory'),
        used_features.index('product_modelname'),
        # used_features.index('customer_country.1'),
        used_features.index('customer_position'),
        used_features.index('response_corporate'),
        used_features.index('expected_timeline'),
        used_features.index('business_area'),
        used_features.index('business_subarea'),
    ],  # train set에서 categorical feature의 indices
)

### 모델 학습

In [30]:
model.fit(
    x_train.fillna(-1), y_train,
    eval_set=(x_val, y_val),
)

Learning rate set to 0.109258
0:	learn: 0.5947608	test: 0.5970619	best: 0.5970619 (0)	total: 92.5ms	remaining: 48s
1:	learn: 0.5234609	test: 0.5275550	best: 0.5275550 (1)	total: 155ms	remaining: 40.3s
2:	learn: 0.4735733	test: 0.4789008	best: 0.4789008 (2)	total: 221ms	remaining: 38.1s
3:	learn: 0.4390563	test: 0.4462146	best: 0.4462146 (3)	total: 291ms	remaining: 37.6s
4:	learn: 0.4111729	test: 0.4188416	best: 0.4188416 (4)	total: 360ms	remaining: 37s
5:	learn: 0.3874550	test: 0.3947183	best: 0.3947183 (5)	total: 421ms	remaining: 36.1s
6:	learn: 0.3717043	test: 0.3794233	best: 0.3794233 (6)	total: 484ms	remaining: 35.5s
7:	learn: 0.3548068	test: 0.3632041	best: 0.3632041 (7)	total: 549ms	remaining: 35.1s
8:	learn: 0.3473126	test: 0.3557995	best: 0.3557995 (8)	total: 611ms	remaining: 34.7s
9:	learn: 0.3357982	test: 0.3454343	best: 0.3454343 (9)	total: 679ms	remaining: 34.6s
10:	learn: 0.3238894	test: 0.3347551	best: 0.3347551 (10)	total: 746ms	remaining: 34.5s
11:	learn: 0.3151901	test

### 모델 성능 보기

In [31]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))

In [32]:
pred = model.predict(x_val.fillna(-1))
pred = np.array([True if x == 'True' else False for x in pred], dtype=bool)
get_clf_eval(y_val, pred)

오차행렬:
 [[  878    92]
 [  511 10379]]

정확도: 0.9492
정밀도: 0.6321
재현율: 0.9052
F1: 0.7444


In [33]:
# def f1_score_str(y_pred, y_val):
#     return f1_score(y_pred, y_val, pos_label='True')

# label_str = df_train['is_converted'].astype(str)

# cv_scores = cross_val_score(model, df_train[used_features].fillna(-1),
#                label_str, cv=StratifiedKFold(n_splits=3, shuffle=True),
#                scoring=make_scorer(f1_score_str))
# print(cv_scores)
# print(cv_scores.mean().round(3))

## 4. 제출하기

### 테스트 데이터 예측

In [34]:
# 예측에 필요한 데이터 분리
x_test = df_test[used_features]

In [35]:
test_pred = model.predict(x_test.fillna(-1))
test_pred = np.array([True if x == 'True' else False for x in test_pred], dtype=bool)
sum(test_pred) # True로 예측된 개수

1651

### 제출 파일 작성

In [36]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["is_converted"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**